In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [51]:
import json
import tensorflow as tf
import numpy as np
from tensorflow.train import BytesList, Feature, FeatureList, Int64List, FloatList
from tensorflow.train import SequenceExample, FeatureLists, Example

from tensorflow.python.lib.io import file_io

# import tensorflow_recommenders as tfrs

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

In [15]:
def string_array(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v.encode('utf-8') for v in value]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v) for v in value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[int(v) for v in value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

## Test - small dataset

In [39]:
def parse_line(ln):
    q_ln = ln['query'] #grabs the query stuff in the json
    c_ln = ln['candidate']
    
    # ===============================
    # Ragged Features - Query
    # ===============================
    ragged_key_list = [
        'track_name_pl',
        'artist_name_pl',
        'album_name_pl',
        # 'track_uri_pl',
        'duration_ms_songs_pl',
        'artist_pop_pl',
        'artists_followers_pl',
        'track_pop_pl',
        'artist_genres_pl',
    ]
    
    ragged_dict = {}
    
    for _ in ragged_key_list:
        ragged_dict[_] = []
        
    for x in q_ln['track_name_pl']:
        ragged_dict['track_name_pl'].append(x.encode('utf8'))
        
    for x in q_ln['artist_name_pl']:
        ragged_dict['artist_name_pl'].append(x.encode('utf8'))
        
    for x in q_ln['album_name_pl']:
        ragged_dict['album_name_pl'].append(x.encode('utf8'))
        
    # for x in q_ln['track_uri_pl']:
    #     ragged_dict['track_uri_pl'].append(x.encode('utf8'))
        
    for x in q_ln['duration_ms_songs_pl']:
        ragged_dict['duration_ms_songs_pl'].append(x)
        
    for x in q_ln['artist_pop_pl']:
        ragged_dict['artist_pop_pl'].append(x)
        
    for x in q_ln['artists_followers_pl']:
        ragged_dict['artists_followers_pl'].append(x)
        
    for x in q_ln['track_pop_pl']:
        ragged_dict['track_pop_pl'].append(x)
        
    for x in q_ln['artist_genres_pl']:
        ragged_dict['artist_genres_pl'].append(x.encode('utf8'))
        
    # Set List Types
    # Bytes
    track_name_pl = BytesList(value=ragged_dict['track_name_pl'])
    artist_name_pl = BytesList(value=ragged_dict['artist_name_pl'])
    album_name_pl = BytesList(value=ragged_dict['album_name_pl'])
    # track_uri_pl = BytesList(value=ragged_dict['track_uri_pl'])
    artist_genres_pl = BytesList(value=ragged_dict['artist_genres_pl'])
    
    # Float List
    duration_ms_songs_pl = FloatList(value=ragged_dict['duration_ms_songs_pl'])
    artist_pop_pl = FloatList(value=ragged_dict['artist_pop_pl'])
    artists_followers_pl = FloatList(value=ragged_dict['artists_followers_pl'])
    track_pop_pl = FloatList(value=ragged_dict['track_pop_pl'])
    
    # Set FeatureLists
    # Bytes
    track_name_pl = FeatureList(feature=[Feature(bytes_list=track_name_pl)])
    artist_name_pl = FeatureList(feature=[Feature(bytes_list=artist_name_pl)])
    album_name_pl = FeatureList(feature=[Feature(bytes_list=album_name_pl)])
    # track_uri_pl = FeatureList(feature=[Feature(bytes_list=track_uri_pl)])
    artist_genres_pl = FeatureList(feature=[Feature(bytes_list=artist_genres_pl)])
    
    # Float Lists
    duration_ms_songs_pl = FeatureList(feature=[Feature(float_list=duration_ms_songs_pl)])
    artist_pop_pl = FeatureList(feature=[Feature(float_list=artist_pop_pl)])
    artists_followers_pl = FeatureList(feature=[Feature(float_list=artists_followers_pl)])
    track_pop_pl = FeatureList(feature=[Feature(float_list=track_pop_pl)])
    
    # ===============================
    # Create Context Features
    # ===============================
    context_features = {
        # playlist - context features
        "name": string_array(q_ln['name']),
        'collaborative' : string_array(q_ln['collaborative']),
        # 'duration_ms_seed_pl' : float_feature(q_ln['duration_ms_seed_pl']),
        'n_songs_pl' : float_feature(q_ln['n_songs_pl']),
        'num_artists_pl' : float_feature(q_ln['num_artists_pl']),
        'num_albums_pl' : float_feature(q_ln['num_albums_pl']),
        'description_pl' : string_array(q_ln['description_pl']),
        
        # seed track - context features
        # 'track_name_seed_track' : string_array(q_ln['track_name_seed_track']),
        # 'artist_name_seed_track' : string_array(q_ln['artist_name_seed_track']),
        # 'album_name_seed_track' : string_array(q_ln['album_name_seed_track']),
        # 'track_uri_seed_track' : string_array(q_ln['track_uri_seed_track']),
        # 'artist_uri_seed_track' : string_array(q_ln['artist_uri_seed_track']),
        # 'album_uri_seed_track' : string_array(q_ln['album_uri_seed_track']),
        # 'duration_seed_track' : float_feature(q_ln['duration_seed_track']),
        # 'track_pop_seed_track' : float_feature(q_ln['track_pop_seed_track']),
        # 'artist_pop_seed_track' : float_feature(q_ln['artist_pop_seed_track']),
        # 'artist_genres_seed_track' : string_array(q_ln['artist_genres_seed_track']),
        # 'artist_followers_seed_track' : float_feature(q_ln['artist_followers_seed_track']),
        
        #candidate features
        "track_name_can": string_array(c_ln['track_name_can']), 
        "artist_name_can": string_array(c_ln['artist_name_can']),
        "album_name_can": string_array(c_ln['album_name_can']),
        "track_uri_can": string_array(c_ln['track_uri_can']),
        # "artist_uri_can": string_array(c_ln['artist_uri_can']),
        # "album_uri_can": string_array(c_ln['album_uri_can']),
        "duration_ms_can": float_feature(c_ln['duration_ms_can']),
        "track_pop_can": float_feature(c_ln['track_pop_can']), 
        "artist_pop_can": float_feature(c_ln['artist_pop_can']),
        "artist_genres_can": string_array(c_ln['artist_genres_can']),
        "artist_followers_can": float_feature(c_ln['artist_followers_can']),
    }
    
    # ===============================
    # Create Sequence
    # ===============================
    seq = SequenceExample(
        context=tf.train.Features(
            feature=context_features
        ),
        feature_lists=FeatureLists(
            feature_list={
                "track_name_pl": track_name_pl,
                "artist_name_pl": artist_name_pl,
                "album_name_pl": album_name_pl,
                # "track_uri_pl": track_uri_pl,
                "duration_ms_songs_pl": duration_ms_songs_pl,
                "artist_pop_pl": artist_pop_pl,
                "artists_followers_pl": artists_followers_pl,
                "track_pop_pl": track_pop_pl,
                "artist_genres_pl": artist_genres_pl
            }
        )
    )
    
    return seq

In [40]:
BUCKET = 'gs://spotify-builtin-2t'
small_SCHEMA_JSON = f'{BUCKET}/schema_small.json'
small_sample_TRAIN_JSON = f'{BUCKET}/train_data_small/000000000000.jsonl'
SMALL_TRAIN_DS = f'{BUCKET}/train_data_small/*.jsonl'

SMALL_DEST = 'gs://spotify-tfrecords-blog/small_data/train'

# Calculate records per file
num_records = sum(1 for _ in file_io.FileIO(small_sample_TRAIN_JSON, 'rb')) #CHANGE THIS TO LARGE DATASET WHEN READY
print("Total number of records: {}".format(num_records))

num_samples = 12228
num_tfrecords = num_records // num_samples 
if num_records % num_samples:
    num_tfrecords += 1

print("Number of Expected TFRecords: {}".format(num_tfrecords))

Total number of records: 32930
Number of Expected TFRecords: 3


### write the records

In [41]:
import time
from tqdm import tqdm

BUCKET = 'gs://spotify-builtin-2t'
# TRAIN_JSON = f'{BUCKET}/train_data_small/*.jsonl'
TRAIN_JSON = f'{BUCKET}/train_data_small/000000000000.jsonl'

GCS_DESTINATION = 'gs://spotify-tfrecords-blog/small_data/train'


record_counter = 0
lns = [] 
tfrec_counter = 0

# quick function to write the data as we read through it
def write_a_tfrec(lns):
    #next write to a tfrecord
    with tf.io.TFRecordWriter(
        GCS_DESTINATION + "/spotify-%.6i-%i.tfrec" % (tfrec_counter, len(lns))
    ) as writer:
        for ln in lns:
            example = parse_line(ln)
            writer.write(example.SerializeToString())
            
            
#record counter makes sure we are batching every `num_samples` and end of list

with file_io.FileIO(TRAIN_JSON, 'r') as reader:
    
    for line in tqdm(reader):
        record_counter += 1
        if record_counter % num_samples == 0 or record_counter == num_records: 
            write_a_tfrec(lns) #write out a batch
            lns = [] #reset to a new batch
            tfrec_counter += 1
        else:
            lns.append(json.loads(line))

32930it [00:33, 985.31it/s] 


In [42]:
#verify records
!gsutil ls $GCS_DESTINATION | head

gs://spotify-tfrecords-blog/small_data/train/spotify_00-12227.tfrec
gs://spotify-tfrecords-blog/small_data/train/spotify_01-12227.tfrec
gs://spotify-tfrecords-blog/small_data/train/spotify_02-8473.tfrec


### confirm the records (read)

In [45]:
from google.cloud import storage

client = storage.Client()

context_features = {
    # playlist - context features
    'name': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'collaborative': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'n_songs_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'num_artists_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'num_albums_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'description_pl': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),

    #candidate features
    'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
}
train_sequence_features = {
    'track_name_pl': tf.io.RaggedFeature(tf.string),
    'artist_name_pl': tf.io.RaggedFeature(tf.string),
    'album_name_pl': tf.io.RaggedFeature(tf.string),
    # 'track_uri_pl': tf.io.RaggedFeature(tf.string),
    'duration_ms_songs_pl': tf.io.RaggedFeature(tf.float32),
    'artist_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artists_followers_pl': tf.io.RaggedFeature(tf.float32),
    'track_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artist_genres_pl': tf.io.RaggedFeature(tf.string),
}
def parse_tfrecord_fn(example):
    example = tf.io.parse_single_sequence_example(
        example, 
        sequence_features=train_sequence_features, 
        context_features=context_features
    )
    return example

train_files= []

BUCKET_NAME_TRAIN = 'spotify-tfrecords-blog'
OBJ_PATH_TRAIN = 'small_data/train'

for blob in client.list_blobs(f'{BUCKET_NAME_TRAIN}', prefix=f'{OBJ_PATH_TRAIN}/', delimiter='/'):
    train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
raw_dataset = tf.data.TFRecordDataset(train_files)

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

parsed_dataset = raw_dataset.map(
        parse_tfrecord_fn,
        num_parallel_calls=tf.data.AUTOTUNE
    ).with_options(options)

In [46]:
parsed_dataset

<_OptionsDataset element_spec=({'album_name_can': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'collaborative': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'description_pl': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'duration_ms_can': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'n_songs_pl': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'name': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'num_albums_pl': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'num_artists_pl': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'track_name_can': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'track_pop_can': TensorSpec(shape=(1,), dtype=tf.float32,

In [48]:
for _ in parsed_dataset.batch(1).take(1):
    print(_)

({'album_name_can': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b"Some Things Don't Come Easy"]], dtype=object)>, 'artist_followers_can': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.]], dtype=float32)>, 'artist_genres_can': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b"'yacht rock'"]], dtype=object)>, 'artist_name_can': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'England Dan Seals']], dtype=object)>, 'artist_pop_can': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[25.]], dtype=float32)>, 'collaborative': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'false']], dtype=object)>, 'description_pl': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'NONE']], dtype=object)>, 'duration_ms_can': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[172240.]], dtype=float32)>, 'n_songs_pl': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[50.]], dtype=float32)>, 'name': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([

## Train (full)

In [66]:
def parse_line(ln):
    q_ln = ln['query'] #grabs the query stuff in the json
    c_ln = ln['candidate']
    
    # ===============================
    # Ragged Features - Query
    # ===============================
    ragged_key_list = [
        'track_name_pl',
        'artist_name_pl',
        'album_name_pl',
        # 'track_uri_pl',
        'duration_ms_songs_pl',
        'artist_pop_pl',
        'artists_followers_pl',
        'track_pop_pl',
        'artist_genres_pl',
    ]
    
    ragged_dict = {}
    
    for _ in ragged_key_list:
        ragged_dict[_] = []
        
    for x in q_ln['track_name_pl']:
        ragged_dict['track_name_pl'].append(x.encode('utf8'))
        
    for x in q_ln['artist_name_pl']:
        ragged_dict['artist_name_pl'].append(x.encode('utf8'))
        
    for x in q_ln['album_name_pl']:
        ragged_dict['album_name_pl'].append(x.encode('utf8'))
        
    # for x in q_ln['track_uri_pl']:
    #     ragged_dict['track_uri_pl'].append(x.encode('utf8'))
        
    for x in q_ln['duration_ms_songs_pl']:
        ragged_dict['duration_ms_songs_pl'].append(x)
        
    for x in q_ln['artist_pop_pl']:
        ragged_dict['artist_pop_pl'].append(x)
        
    for x in q_ln['artists_followers_pl']:
        ragged_dict['artists_followers_pl'].append(x)
        
    for x in q_ln['track_pop_pl']:
        ragged_dict['track_pop_pl'].append(x)
        
    for x in q_ln['artist_genres_pl']:
        ragged_dict['artist_genres_pl'].append(x.encode('utf8'))
        
    # Set List Types
    # Bytes
    track_name_pl = BytesList(value=ragged_dict['track_name_pl'])
    artist_name_pl = BytesList(value=ragged_dict['artist_name_pl'])
    album_name_pl = BytesList(value=ragged_dict['album_name_pl'])
    # track_uri_pl = BytesList(value=ragged_dict['track_uri_pl'])
    artist_genres_pl = BytesList(value=ragged_dict['artist_genres_pl'])
    
    # Float List
    duration_ms_songs_pl = FloatList(value=ragged_dict['duration_ms_songs_pl'])
    artist_pop_pl = FloatList(value=ragged_dict['artist_pop_pl'])
    artists_followers_pl = FloatList(value=ragged_dict['artists_followers_pl'])
    track_pop_pl = FloatList(value=ragged_dict['track_pop_pl'])
    
    # Set FeatureLists
    # Bytes
    track_name_pl = FeatureList(feature=[Feature(bytes_list=track_name_pl)])
    artist_name_pl = FeatureList(feature=[Feature(bytes_list=artist_name_pl)])
    album_name_pl = FeatureList(feature=[Feature(bytes_list=album_name_pl)])
    # track_uri_pl = FeatureList(feature=[Feature(bytes_list=track_uri_pl)])
    artist_genres_pl = FeatureList(feature=[Feature(bytes_list=artist_genres_pl)])
    
    # Float Lists
    duration_ms_songs_pl = FeatureList(feature=[Feature(float_list=duration_ms_songs_pl)])
    artist_pop_pl = FeatureList(feature=[Feature(float_list=artist_pop_pl)])
    artists_followers_pl = FeatureList(feature=[Feature(float_list=artists_followers_pl)])
    track_pop_pl = FeatureList(feature=[Feature(float_list=track_pop_pl)])
    
    # ===============================
    # Create Context Features
    # ===============================
    context_features = {
        # playlist - context features
        "name": string_array(q_ln['name']),
        'collaborative' : string_array(q_ln['collaborative']),
        # 'duration_ms_seed_pl' : float_feature(q_ln['duration_ms_seed_pl']),
        'n_songs_pl' : float_feature(q_ln['n_songs_pl']),
        'num_artists_pl' : float_feature(q_ln['num_artists_pl']),
        'num_albums_pl' : float_feature(q_ln['num_albums_pl']),
        'description_pl' : string_array(q_ln['description_pl']),
        
        # seed track - context features
        'track_name_seed_track' : string_array(q_ln['track_name_seed_track']),
        'artist_name_seed_track' : string_array(q_ln['artist_name_seed_track']),
        'album_name_seed_track' : string_array(q_ln['album_name_seed_track']),
        # 'track_uri_seed_track' : string_array(q_ln['track_uri_seed_track']),
        # 'artist_uri_seed_track' : string_array(q_ln['artist_uri_seed_track']),
        # 'album_uri_seed_track' : string_array(q_ln['album_uri_seed_track']),
        'duration_seed_track' : float_feature(q_ln['duration_seed_track']),
        'track_pop_seed_track' : float_feature(q_ln['track_pop_seed_track']),
        'artist_pop_seed_track' : float_feature(q_ln['artist_pop_seed_track']),
        'artist_genres_seed_track' : string_array(q_ln['artist_genres_seed_track']),
        'artist_followers_seed_track' : float_feature(q_ln['artist_followers_seed_track']),
        
        #candidate features
        "track_name_can": string_array(c_ln['track_name_can']), 
        "artist_name_can": string_array(c_ln['artist_name_can']),
        "album_name_can": string_array(c_ln['album_name_can']),
        "track_uri_can": string_array(c_ln['track_uri_can']),
        # "artist_uri_can": string_array(c_ln['artist_uri_can']),
        # "album_uri_can": string_array(c_ln['album_uri_can']),
        "duration_ms_can": float_feature(c_ln['duration_ms_can']),
        "track_pop_can": float_feature(c_ln['track_pop_can']), 
        "artist_pop_can": float_feature(c_ln['artist_pop_can']),
        "artist_genres_can": string_array(c_ln['artist_genres_can']),
        "artist_followers_can": float_feature(c_ln['artist_followers_can']),
    }
    
    # ===============================
    # Create Sequence
    # ===============================
    seq = SequenceExample(
        context=tf.train.Features(
            feature=context_features
        ),
        feature_lists=FeatureLists(
            feature_list={
                "track_name_pl": track_name_pl,
                "artist_name_pl": artist_name_pl,
                "album_name_pl": album_name_pl,
                # "track_uri_pl": track_uri_pl,
                "duration_ms_songs_pl": duration_ms_songs_pl,
                "artist_pop_pl": artist_pop_pl,
                "artists_followers_pl": artists_followers_pl,
                "track_pop_pl": track_pop_pl,
                "artist_genres_pl": artist_genres_pl
            }
        )
    )
    
    return seq

### Configure record count

* original jsonl files have ~`13297` samples per file

In [72]:
BUCKET = 'gs://spotify-builtin-2t'
SCHEMA_JSON = f'{BUCKET}/schema.json' # schema_small.json
TRAIN_JSON = f'{BUCKET}/train_data/*.jsonl' # 000000000000.jsonl
TRAIN_SAMPLE_FILE = 'gs://spotify-builtin-2t/train_data/000000000001.jsonl'
# spotify-builtin-2t/candidate_songs

TF_RECORDS_DIR = 'gs://spotify-tfrecords-blog/train_v2'

BUCKET_NAME_TRAIN='spotify-builtin-2t'
OBJ_PATH_TRAIN='train_data'

files = []

for blob in client.list_blobs(f'{BUCKET_NAME_TRAIN}', prefix=f'{OBJ_PATH_TRAIN}/', delimiter='/'):
    files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
len(files)

In [76]:
total_samples = 0
# num_samples = 0

for file in files:
    num_samples = 0
    # Calculate records per file
    num_samples = sum(1 for _ in file_io.FileIO(file, 'rb'))
    total_samples = total_samples+num_samples

print("Total number of samples: {}".format(total_samples))

samples_per_file = 12228

num_tfrecords = total_samples // samples_per_file 

if num_records % total_samples:
    num_tfrecords += 1

print("Number of Expected TFRecords: {}".format(num_tfrecords))

KeyboardInterrupt: 

In [75]:
# total = 0
# num_list = [1,2,3,4,5,6,7]

# for num in num_list:
#     total = total+num
#     print(f'Loop {num} total:{total}')

Loop 1 total:1
Loop 2 total:3
Loop 3 total:6
Loop 4 total:10
Loop 5 total:15
Loop 6 total:21
Loop 7 total:28


In [ ]:
# quick function to write the data as we read through it
def write_a_tfrec(lns):
    #next write to a tfrecord
    with tf.io.TFRecordWriter(
        GCS_DESTINATION + "/spotify-%.6i-%i.tfrec" % (tfrec_counter, len(lns))
    ) as writer:
        for ln in lns:
            example = parse_line(ln)
            writer.write(example.SerializeToString())

for file in files:
    with file_io.FileIO(file, 'r') as reader:
        for line in tqdm(reader):
            record_counter += 1
            if record_counter % num_samples == 0 or record_counter == num_records: 
                write_a_tfrec(lns) #write out a batch
                lns = [] #reset to a new batch
                tfrec_counter += 1
            else:
                lns.append(json.loads(line))

## Candidate DS

In [55]:
def parse_line(ln):
    c_ln = ln['candidate']

    # ===============================
    # Create Context Features
    # ===============================
    context_features = { 
        #candidate features
        "track_name_can": string_array(c_ln['track_name_can']), 
        "artist_name_can": string_array(c_ln['artist_name_can']),
        "album_name_can": string_array(c_ln['album_name_can']),
        "track_uri_can": string_array(c_ln['track_uri_can']),
        # "artist_uri_can": string_array(c_ln['artist_uri_can']),
        # "album_uri_can": string_array(c_ln['album_uri_can']),
        "duration_ms_can": float_feature(c_ln['duration_ms_can']),
        "track_pop_can": float_feature(c_ln['track_pop_can']), 
        "artist_pop_can": float_feature(c_ln['artist_pop_can']),
        "artist_genres_can": string_array(c_ln['artist_genres_can']),
        "artist_followers_can": float_feature(c_ln['artist_followers_can']),
    }
    
    ex = Example(
        features=tf.train.Features(
            feature=context_features
        )
    )
    
    return ex

In [56]:
BUCKET_NAME_TRAIN='spotify-builtin-2t'
OBJ_PATH_TRAIN='candidate_songs'

files = []

for blob in client.list_blobs(f'{BUCKET_NAME_TRAIN}', prefix=f'{OBJ_PATH_TRAIN}/', delimiter='/'):
    files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
files

['gs://spotify-builtin-2t/candidate_songs/000000000000.jsonl',
 'gs://spotify-builtin-2t/candidate_songs/000000000001.jsonl',
 'gs://spotify-builtin-2t/candidate_songs/000000000002.jsonl']

In [63]:
BUCKET = 'gs://spotify-builtin-2t'
candidate_files = f'{BUCKET}/candidate_songs/*.jsonl' # TRAIN_JSON

GCS_DESTINATION = 'gs://spotify-tfrecords-blog/candidate_tracks_v2'

record_counter = 0
lns = [] 
tfrec_counter = 0

# quick function to write the data as we read through it
def write_a_tfrec(lns):
    #next write to a tfrecord
    with tf.io.TFRecordWriter(
        GCS_DESTINATION + "/spotify_%.6i-%i.tfrec" % (tfrec_counter, len(lns))
    ) as writer:
        for ln in lns:
            example = parse_line(ln)
            writer.write(example.SerializeToString())

for file in files:
    with file_io.FileIO(file, 'r') as reader:
        for line in tqdm(reader):
            record_counter += 1
            if record_counter % num_samples == 0 or record_counter == num_records: 
                write_a_tfrec(lns) #write out a batch
                lns = [] #reset to a new batch
                tfrec_counter += 1
            else:
                lns.append(json.loads(line))

748949it [01:41, 7408.34it/s] 
750354it [01:39, 7524.24it/s] 
750258it [01:38, 7652.32it/s] 


### Confirm - read records

In [64]:
from google.cloud import storage

client = storage.Client()

context_features = {
    #candidate features
    'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
}

def parse_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=context_features
    )
    return example

train_files= []

BUCKET_NAME_TRAIN = 'spotify-tfrecords-blog'
OBJ_PATH_TRAIN = 'candidate_tracks_v2'
# file = 'gs://spotify-tfrecords-blog/candidate_tracks/spotify_000100-12227.tfrec'

for blob in client.list_blobs(f'{BUCKET_NAME_TRAIN}', prefix=f'{OBJ_PATH_TRAIN}/', delimiter='/'):
    train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
raw_dataset = tf.data.TFRecordDataset(train_files)

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

parsed_dataset = raw_dataset.map(
        parse_tfrecord_fn,
        num_parallel_calls=tf.data.AUTOTUNE
    ).with_options(options)

In [65]:
cnt = parsed_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=2237540>

In [62]:
# for _ in parsed_dataset.batch(35).take(1):
#     print(_)

## Archive

In [31]:
# !gsutil cp gs://spotify-builtin-2t/train_data/000000000000.jsonl train_test.jsonl
# !gsutil cp gs://spotify-builtin-2t/candidate_songs/000000000000.jsonl candidate_test.jsonl

# !gsutil cp gs://spotify-builtin-2t/schema.json .
# !gsutil cp gs://spotify-builtin-2t/schema_small.json .

Copying gs://spotify-builtin-2t/schema.json...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operation completed over 1 objects/1.6 KiB.                                      
Copying gs://spotify-builtin-2t/schema_small.json...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


In [ ]:
# small_SCHEMA_JSON = f'{BUCKET}/schema_small.json'
# small_TRAIN_JSON = f'{BUCKET}/train_data_small/000000000000.jsonl

In [ ]:
# test_files = ['gs://limestone-recsys/tf-record-all-features/2022-05-05174504/file_00-12227.tfrec']

# sequence_features_ = {
#      "last_viewed": tf.io.RaggedFeature(tf.string),
#        "productTypeCombo_ss": tf.io.RaggedFeature(tf.string),
#        "Searchable_t": tf.io.RaggedFeature(tf.string),
#        "spellcheck": tf.io.RaggedFeature(tf.string)
# }

# context_features_ = {
#         "query": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "IVM_s": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "description": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "total_ratings_i": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)), 
#         "overall_ratings": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)), 
#         "avg_rating_td": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)), 
#         "parent_description": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "Brand_s": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "item_type": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "prc_rdc_amt": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)), 
#         "quantity_sold": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)), 
#         "sales_dollar_f": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),  
#         "freight_term": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "is_energy_star_s": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "price_td": tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)), 
#         "PriceRange_s": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "clean_Brand_s": tf.io.FixedLenFeature(dtype=tf.string, shape=(1)), 
#         "visual": tf.io.FixedLenFeature(dtype=tf.float32, shape=(2048)), 
#         "month": tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
#         "hour": tf.io.FixedLenFeature(dtype=tf.int64, shape=(1))
#     }

# def parse_tfrecord_fn(example):
#     example = tf.io.parse_single_sequence_example(example, sequence_features=sequence_features_, context_features=context_features_)
#     return example

# raw_test_ds = tf.data.TFRecordDataset(test_files)

# options = tf.data.Options()
# options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

# parsed_test_ds = raw_test_ds.map(
#         parse_tfrecord_fn,
#         num_parallel_calls=tf.data.AUTOTUNE).with_options(options)

# def return_tensors(context, sequence):
#         a = sequence['productTypeCombo_ss'].to_tensor(default_value='', shape=[None, 8])
#         b = sequence['last_viewed'].to_tensor(default_value='', shape=[None, 3])
#         context2 = context.copy()
#         context2['productTypeCombo_ss'] = a
#         context2['last_viewed'] = b
#         return context2
    
# parsed_test_ds_2 = parsed_test_ds.map(return_tensors, num_parallel_calls=tf.data.AUTOTUNE)

# for _ in parsed_test_ds_2.batch(1).take(1):
#     print(_)

In [ ]:
# for _ in parsed_test_ds.batch(1).take(1):
    # print(_)
    
for raw_record in raw_test_ds.take(5):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)